# Evaluating a Linear Model

MSE is a useful metric for seeing the performance of a model, but other metrics can help us decide which model to use.

Here we will use **statsmodels** for fitting linear models since the package easily computes the metrics I want to see.

Let's load in the Boston housing dataset.

In [1]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

In [2]:
boston_obj = load_boston()
data_train, data_test, price_train, price_test = train_test_split(boston_obj.data, boston_obj.target)

## Fitting a Linear Model with OLS in **statsmodels**

The `OLS` object in **statsmodels** handles fitting models with OLS. Below I show how to fit the same model for Boston home prices I fitted using **scikit-learn**.

In [3]:
import statsmodels.api as sm
import numpy as np

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [4]:
data_train, data_test = sm.add_constant(data_train), sm.add_constant(data_test)    # Necessary to add the intercept
data_train[:5, :]

array([[  1.00000000e+00,   2.98190000e-01,   0.00000000e+00,
          6.20000000e+00,   0.00000000e+00,   5.04000000e-01,
          7.68600000e+00,   1.70000000e+01,   3.37510000e+00,
          8.00000000e+00,   3.07000000e+02,   1.74000000e+01,
          3.77510000e+02,   3.92000000e+00],
       [  1.00000000e+00,   9.16400000e-02,   0.00000000e+00,
          1.08100000e+01,   0.00000000e+00,   4.13000000e-01,
          6.06500000e+00,   7.80000000e+00,   5.28730000e+00,
          4.00000000e+00,   3.05000000e+02,   1.92000000e+01,
          3.90910000e+02,   5.52000000e+00],
       [  1.00000000e+00,   2.77974000e+00,   0.00000000e+00,
          1.95800000e+01,   0.00000000e+00,   8.71000000e-01,
          4.90300000e+00,   9.78000000e+01,   1.34590000e+00,
          5.00000000e+00,   4.03000000e+02,   1.47000000e+01,
          3.96900000e+02,   2.92900000e+01],
       [  1.00000000e+00,   4.26131000e+00,   0.00000000e+00,
          1.81000000e+01,   0.00000000e+00,   7.70000000e-0

In [5]:
data_train[:5, 0]

array([ 1.,  1.,  1.,  1.,  1.])

In [6]:
ols1 = sm.OLS(price_train, data_train)    # Target, features
model1 = ols1.fit()
model1.params    # The parameters of the regression

array([  4.55500487e+01,  -1.15881758e-01,   6.46705142e-02,
        -2.94914566e-02,   2.56042181e+00,  -1.86435260e+01,
         2.84961332e+00,   3.75427007e-03,  -1.62616270e+00,
         3.11254205e-01,  -1.31510778e-02,  -1.01784717e+00,
         6.47903897e-03,  -4.89312934e-01])

In [7]:
model1.predict([[    # An example prediction
    1,      # Intercept term; always 1
    10,     # Per capita crime rate
    25,     # Proportion of land zoned for large homes
    5,      # Proportion of land zoned for non-retail business
    1,      # Tract bounds the Charles River
    0.3,    # NOX concentration
    10,     # Average number of rooms per dwelling
    2,      # Proportion of owner-occupied units built prior to 1940
    10,     # Weighted distance to employment centers
    3,      # Index for highway accessibility
    400,    # Tax rate
    15,     # Pupil/teacher ratio
    200,    # Index for number of blacks
    5       # % lower status of population
]])

array([ 34.32478248])

We can get a summary of the model easily in **statsmodels**.

In [8]:
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.736
Model:                            OLS   Adj. R-squared:                  0.727
Method:                 Least Squares   F-statistic:                     78.28
Date:                Mon, 23 Apr 2018   Prob (F-statistic):           5.36e-97
Time:                        11:03:48   Log-Likelihood:                -1110.4
No. Observations:                 379   AIC:                             2249.
Df Residuals:                     365   BIC:                             2304.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         45.5500      5.891      7.732      0.0

Let's parse these results:

* `R-squared` (mathematically, $R^2$) describes how much variation in the target variable the model is able to "explain." (Think: predict.) Practitioners prefer `Adj. R-squared` ($\bar{R}^2$) since this takes into account how many variables are used. (it is impossible for $R^2$ to go down when adding variables even if those variables only contribute noise; $\bar{R}^2$ doesn't have this property.) Here $\bar{R}^2$ is somewhat high, suggesting that the model does a reasonable job at predicting home prices.
* `F-statistic` is the test statistic for a statistical test to determine if any coefficients in the model are not zero. `Prob (F-statistic)` is the corresponding $p$-value. Here the model clearly has a non-zero coefficient, though the statistic does not say which.
* `AIC` is the **Akaike information criterion (AIC)**, and `BIC` the **Bayesian information criterion (BIC)**. These are measures of the quality of statistical models and provide a means to decide between models. Models that lead to smaller AIC and BIC are seen as better.
* The table contains the coefficients of the statistical model and the results of $t$-tests to determine if the coefficients are zero or not, in addition to confidence intervals for the coefficient values. We might consider removing features with coefficients not statistically different from zero from our model (but we should also refer to the AIC and BIC statistics when making decisions between models).

## Using AIC to Pick Models

Let's see how we can use the AIC to decide between different models. (The BIC can be used similarly.) Notice that in our table the third and seventh features don't have coefficients statistically different from zero (these correspond to proportion of non-retail business acres per town and proportion of owner-occupied units built prior to 1940). Does a model without these features do better according to the AIC?

In [9]:
ols2 = sm.OLS(price_train, np.delete(data_train, [3, 7], axis=1))
model2 = ols2.fit()
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.736
Model:                            OLS   Adj. R-squared:                  0.728
Method:                 Least Squares   F-statistic:                     92.95
Date:                Mon, 23 Apr 2018   Prob (F-statistic):           6.43e-99
Time:                        11:10:38   Log-Likelihood:                -1110.6
No. Observations:                 379   AIC:                             2245.
Df Residuals:                     367   BIC:                             2292.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         45.5559      5.829      7.816      0.0

This model has a smaller AIC.

The different AIC values can be interpreted this way: If model 1 has $\text{AIC}_1$ and model 2 $\text{AIC}_2$ and $\text{AIC}_2 < \text{AIC}_1$, then model 1 is $\exp((\text{AIC}_2 - \text{AIC}_1)/2)$ times more likely to minimize information loss than model 2.

In [ ]:
np.exp((model2.aic - model1.aic)/2)

The inverse of that quantity can be interpreted as how much more likely model 2 is to minimize the information loss than model 1.

In [ ]:
np.exp((model1.aic - model2.aic)/2)

So we can see that model 2 should be preferred to model 1.

Let's see how model 2 would have done on the test set, evaluating its performance with the MSE.

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
price_train_pred = model2.predict(np.delete(data_train, [3, 7], axis=1))
mean_squared_error(price_train, price_train_pred)     # Performance on the training set

In [ ]:
price_test_pred = model2.predict(np.delete(data_test, [3, 7], axis=1))
mean_squared_error(price_test, price_test_pred)     # Performance on the training set

In comparison to model 1...

In [ ]:
price_test_pred_mod1 = model1.predict(data_test)
mean_squared_error(price_test, price_test_pred_mod1)

Model 2 did better.